"The spectral centroid indicates at which frequency the energy of a spectrum is centred upon" (Chauhan, 2020)

Breebaart (2004) compared various features such as mel-frequency cepstral coefficients (MFCC), auditory filter-bank temporal envelope (AFTE), standard low-level (SLL) and psychoacoustic (PA) features when training a standard Gaussian framework for both genre and general audio classification.

**AFTE features**: GammaTone filter banks.

**SLL features**: Root-mean-square (RMS) level, spectral (centroid, bandwidth, and roll-off), zero-crossing rate, band energy ratio, delta spectrum magnitude, and pitch.

**PA features**: Roughness (the perception of temporal envelope modulations in the range of about 20-150 Hz), loudness (the sensation of signal strength) and sharpness (the spectral density and the relative strength of high-frequency energy).

It was demonstrated that AFTE features provided the highest accuracy in Breebaart's study with MFCC's coming in at a close second, this corresponds with Liu's (2018) study showing 79% accuracy when training an attention based long short-term memory (LSTM+A) network with GFCC's.

**GFCC's** and GammaTone filter bands tend to give a better resolution in the lower frequency ranges and provide a Cochlegram (relatively to Cochlea, a component of the inner ear)


The literature above shows the results for general audio classification, whereas Deng (2008) and Racharla et. al (2020) showed that MFCC and perception-based features dominated in SVM's for instrument classification models with up to 79% accuracy.

**MFCC's** represent a non-linear 'spectrum-of-a-spectrum' where "the frequency bands are equally spaced on the mel scale, which approximates the human auditory system’s response" (Sdour 2020)

Rathikarani (2020) shows 98% accuracy with SVM and 95% with kNN classifiers using MFCC features in instrument classification.

SVC kNN  best : MLP MFCC & GFCC

## Convert .bib to Harvard
Zaman, K. et al. (2023) ‘A Survey of Audio Classification Using Deep Learning’, IEEE Access, 11, pp. 106620–106649. doi: 10.1109/ACCESS.2023.3318015.


Racharla, K. et al. (2020) ‘Predominant Musical Instrument Classification based on Spectral Features’, in 2020 7th International Conference on Signal Processing and Integrated Networks (SPIN). IEEE. doi: 10.1109/spin48934.2020.9071125.

